In [29]:
import cv2
import numpy as np
import os

# List of coral images without extensions
coral_images_names = ['coral1png', 'coral2png', 'coral3png', 'coral4png', 'coral5png', 'coral6png']
coral_images_paths = [f"D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/{name}.png" for name in coral_images_names]

reference_image_path = 'D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/OneYearImage.jpg'
pink_mask_path = 'D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/PINK_MASK.png'
white_mask_path = 'D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/WHITE_MASK.png'

# Load reference images and masks
reference_image = cv2.imread(reference_image_path)
pink_mask = cv2.imread(pink_mask_path, cv2.IMREAD_GRAYSCALE)
white_mask = cv2.imread(white_mask_path, cv2.IMREAD_GRAYSCALE)

# Verify if reference images and masks are loaded successfully
if reference_image is None or pink_mask is None or white_mask is None:
    raise ValueError("Error loading reference images or masks. Check file paths.")

# Resize the reference image to match the mask dimensions
reference_image_resized = cv2.resize(reference_image, (pink_mask.shape[1], pink_mask.shape[0]))

# Loop through each coral image to analyze changes
for coral_path in coral_images_paths:
    # Load current coral image
    current_image = cv2.imread(coral_path)

    if current_image is None:
        print(f"Error loading image {coral_path}. Skipping...")
        continue

    # Resize current image to match mask size
    current_image_resized = cv2.resize(current_image, (pink_mask.shape[1], pink_mask.shape[0]))

    # Detect growth in pink and white masked areas
    growth_pink = cv2.bitwise_and(current_image_resized, current_image_resized, mask=pink_mask)
    reference_pink = cv2.bitwise_and(reference_image_resized, reference_image_resized, mask=pink_mask)
    growth_difference_pink = cv2.absdiff(growth_pink, reference_pink)

    growth_white = cv2.bitwise_and(current_image_resized, current_image_resized, mask=white_mask)
    reference_white = cv2.bitwise_and(reference_image_resized, reference_image_resized, mask=white_mask)
    growth_difference_white = cv2.absdiff(growth_white, reference_white)

    # Combine growth differences for pink and white areas
    total_growth_difference = cv2.addWeighted(growth_difference_pink, 1, growth_difference_white, 1, 0)

    # Convert to grayscale and apply threshold to highlight contours
    total_growth_gray = cv2.cvtColor(total_growth_difference, cv2.COLOR_BGR2GRAY)
    _, total_growth_thresh = cv2.threshold(total_growth_gray, 50, 255, cv2.THRESH_BINARY)

    # Find contours of the detected changes
    contours, _ = cv2.findContours(total_growth_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 50:  # Filter small noise by contour area
            cv2.drawContours(current_image_resized, [contour], -1, (0, 255, 0), 2)  # Green color for growth

    # Display the result for each coral image
    window_name = f"Coral Health Analysis - {os.path.basename(coral_path)}"
    cv2.imshow(window_name, current_image_resized)
    cv2.waitKey(1000)  # Display each image for half a second

    # Optionally save the result with contours for later analysis
    output_path = f"D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/results/{os.path.basename(coral_path)}_analysis.png"
    cv2.imwrite(output_path, current_image_resized)
    print(f"Analysis saved to {output_path}")

cv2.destroyAllWindows()


Analysis saved to D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/results/coral1png.png_analysis.png
Analysis saved to D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/results/coral2png.png_analysis.png
Analysis saved to D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/results/coral3png.png_analysis.png
Analysis saved to D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/results/coral4png.png_analysis.png
Analysis saved to D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/results/coral5png.png_analysis.png
Analysis saved to D:/Vortex/ROV/Phase 2/computer vision/Tasks2025/task 2/results/coral6png.png_analysis.png
